In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Importing the Dataset
df = pd.read_csv("dataset.csv")
df.head(3)


,identifierHash,type,country,language,socialNbFollowers,socialNbFollows,socialProductsLiked,productsListed,productsSold,productsPassRate,...,civilityTitle,hasAnyApp,hasAndroidApp,hasIosApp,hasProfilePicture,daysSinceLastLogin,seniority,seniorityAsMonths,seniorityAsYears,countryCode
0,-7279641312655250028,user,Etats-Unis,en,3,8,0,0,0,0.0,...,mrs,False,False,False,True,709,3205,106.83,8.9,us
1,-1456013578740053406,user,Allemagne,de,3,8,0,0,0,0.0,...,mrs,False,False,False,True,709,3205,106.83,8.9,de
2,9006282053848196165,user,Suède,en,3,8,0,0,0,0.0,...,mr,True,False,True,True,689,3205,106.83,8.9,se


In [3]:
df.dtypes

identifierHash           int64
type                    object
country                 object
language                object
socialNbFollowers        int64
socialNbFollows          int64
socialProductsLiked      int64
productsListed           int64
productsSold             int64
productsPassRate       float64
productsWished           int64
productsBought           int64
gender                  object
civilityGenderId         int64
civilityTitle           object
hasAnyApp                 bool
hasAndroidApp             bool
hasIosApp                 bool
hasProfilePicture         bool
daysSinceLastLogin       int64
seniority                int64
seniorityAsMonths      float64
seniorityAsYears       float64
countryCode             object
dtype: object

In [4]:
# Create DF with required cols
df_1 = df[["hasProfilePicture","country","language","gender","hasAndroidApp","hasIosApp","daysSinceLastLogin","productsListed","productsSold","productsPassRate","productsWished","productsBought","socialProductsLiked","socialNbFollowers","socialNbFollows" ]]

In [5]:
# Check Profile Pic value counts
(df_1["hasProfilePicture"]).value_counts()

True     97018
False     1895
Name: hasProfilePicture, dtype: int64

In [6]:
# Dropping profile pic, as its one sided
df = df_1.drop(columns="hasProfilePicture")
df.head(2)

,country,language,gender,hasAndroidApp,hasIosApp,daysSinceLastLogin,productsListed,productsSold,productsPassRate,productsWished,productsBought,socialProductsLiked,socialNbFollowers,socialNbFollows
0,Etats-Unis,en,F,False,False,709,0,0,0.0,0,0,0,3,8
1,Allemagne,de,F,False,False,709,0,0,0.0,0,0,0,3,8


In [7]:
# Create Active Users Filter, based on IF: product sold or bought, 
df_active = df.loc[(df["productsListed"] !=0)|(df["productsSold"] !=0)|(df["productsWished"]!=0)|(df["productsBought"]!=0)|(df["socialProductsLiked"]!=0)|(df["socialNbFollows"]>8)]


In [8]:
# Checking for above, the threshold of Follows.
(df["socialNbFollows"]<8).value_counts()

False    98774
True       139
Name: socialNbFollows, dtype: int64

In [9]:
# Assigining new column based on APP Type as "OS"
df = df.assign(OS=np.where((df["hasAndroidApp"]&df["hasIosApp"]==True),"Both Apps",(np.where((df["hasAndroidApp"]==True),"Android",(np.where((df["hasIosApp"]==True),"iOS","Browser"))))))

In [10]:
# Checking Values for OS Types
df["OS"].value_counts()

Browser      72739
iOS          21355
Android       4647
Both Apps      172
Name: OS, dtype: int64

In [11]:
# Rerun Active Users with new included OS
df_active = df.loc[(df["productsListed"] !=0)|(df["productsSold"] !=0)|(df["productsWished"]!=0)|(df["productsBought"]!=0)|(df["socialProductsLiked"]!=0)|(df["socialNbFollows"]>8)]


In [12]:
# Checking values for active OS Types
df_active["OS"].value_counts()

Browser      12047
iOS           8638
Android       1469
Both Apps      134
Name: OS, dtype: int64

In [13]:
# Creating CSV to check in Tableau for any differences compared to previous ALL Dataframe
df_active.to_csv("active_users_brian.csv")

In [14]:
# Creating counter check DF with different Criteria
df_active_julien = df.loc[((df['socialProductsLiked'] !=0) | (df['productsListed'] !=0)) | (df['productsBought'] !=0) | (df['productsSold'] !=0) | (df["productsWished"] != 0)]

In [15]:
# df_active_julien.to_csv("active_users_julien.csv")


In [16]:
# Segmenting Buyers and Sellers - Creating ratios, in order to determine category
df_active = df_active.assign(ratiosellbuy=df_active["productsSold"]/df_active["productsBought"]).assign(ratiobuysell=df_active["productsBought"]/df_active["productsSold"])


In [17]:
df_active.loc[(df_active["productsSold"]>0)&(df_active["productsListed"]>0)]

,country,language,gender,hasAndroidApp,hasIosApp,daysSinceLastLogin,productsListed,productsSold,productsPassRate,productsWished,productsBought,socialProductsLiked,socialNbFollowers,socialNbFollows,OS,ratiosellbuy,ratiobuysell
46,France,fr,F,False,True,127,1,1,0.0,0,1,0,31,8,iOS,1.00,1.0
133,France,fr,F,False,True,485,1,1,0.0,0,0,6,7,8,iOS,inf,0.0
614,Finlande,en,F,False,True,82,4,3,100.0,0,0,0,8,8,iOS,inf,0.0
1054,Pays-Bas,en,F,False,True,377,2,2,100.0,17,8,33,29,8,iOS,0.25,4.0
1169,Italie,it,F,False,True,15,3,1,0.0,1,0,1,26,9,iOS,inf,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98408,Italie,it,F,True,True,11,20,27,85.0,0,0,3,21,9,Both Apps,inf,0.0
98417,France,fr,M,False,False,133,1,1,0.0,0,0,0,3,8,Browser,inf,0.0
98442,Grèce,en,F,False,True,17,3,1,0.0,9,1,14,31,7,iOS,1.00,1.0
98501,Etats-Unis,en,M,False,False,293,102,63,98.0,3,0,5,54,8,Browser,inf,0.0


In [18]:
df_active.loc[((df_active["ratiosellbuy"]<3)&(df_active["ratiosellbuy"]>0))|((df_active["ratiosellbuy"]<3)&(df_active["ratiobuysell"]>0)),"productsListed"].describe()

count    5313.000000
mean        0.305101
std         3.505064
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max       202.000000
Name: productsListed, dtype: float64

In [19]:
df_active.loc[((df_active["ratiosellbuy"]<3)|((df_active["ratiosellbuy"]<3)&(df_active["ratiobuysell"]>0))&(df_active["ratiosellbuy"]>0))&(df_active["productsListed"]>=1),"productsListed"].describe()

count    357.000000
mean       4.540616
std       12.807418
min        1.000000
25%        1.000000
50%        2.000000
75%        4.000000
max      202.000000
Name: productsListed, dtype: float64

In [20]:
df_active.loc[df_active["ratiosellbuy"]==np.inf]

,country,language,gender,hasAndroidApp,hasIosApp,daysSinceLastLogin,productsListed,productsSold,productsPassRate,productsWished,productsBought,socialProductsLiked,socialNbFollowers,socialNbFollows,OS,ratiosellbuy,ratiobuysell
133,France,fr,F,False,True,485,1,1,0.0,0,0,6,7,8,iOS,inf,0.0
164,Royaume-Uni,en,F,False,True,665,0,3,66.0,0,0,0,10,8,iOS,inf,0.0
242,France,fr,M,False,False,405,0,1,0.0,0,0,0,3,8,Browser,inf,0.0
614,Finlande,en,F,False,True,82,4,3,100.0,0,0,0,8,8,iOS,inf,0.0
1084,Allemagne,en,F,False,True,192,0,2,0.0,0,0,1,5,8,iOS,inf,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98417,France,fr,M,False,False,133,1,1,0.0,0,0,0,3,8,Browser,inf,0.0
98501,Etats-Unis,en,M,False,False,293,102,63,98.0,3,0,5,54,8,Browser,inf,0.0
98558,Royaume-Uni,en,F,False,False,706,0,1,0.0,0,0,0,5,8,Browser,inf,0.0
98690,Italie,it,F,True,False,684,0,1,0.0,0,0,1,11,9,Android,inf,0.0


In [21]:
df_active.loc[df_active["ratiobuysell"]==np.inf]

,country,language,gender,hasAndroidApp,hasIosApp,daysSinceLastLogin,productsListed,productsSold,productsPassRate,productsWished,productsBought,socialProductsLiked,socialNbFollowers,socialNbFollows,OS,ratiosellbuy,ratiobuysell
51,Etats-Unis,en,M,False,False,26,0,0,0.0,14,1,1,3,9,Browser,0.0,inf
85,Suède,en,M,False,False,543,0,0,0.0,107,1,0,5,8,Browser,0.0,inf
95,Canada,en,F,True,False,208,0,0,0.0,7,1,1,4,10,Android,0.0,inf
102,Royaume-Uni,en,F,False,False,27,0,0,0.0,0,2,0,3,8,Browser,0.0,inf
114,Etats-Unis,en,M,False,True,15,0,0,0.0,4,6,0,28,29,iOS,0.0,inf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98853,Italie,it,M,False,False,148,0,0,0.0,0,5,7,4,10,Browser,0.0,inf
98865,Allemagne,de,M,False,False,247,0,0,0.0,3,11,6,4,8,Browser,0.0,inf
98879,Danemark,en,F,False,False,204,0,0,0.0,3,1,1,5,8,Browser,0.0,inf
98886,Italie,it,F,False,True,24,0,0,0.0,153,22,13,13,12,iOS,0.0,inf


In [22]:
df_active.loc[(df_active["ratiosellbuy"]>1.5)&(df_active["ratiosellbuy"]!=np.inf)&(df_active["productsSold"]>5)]

,country,language,gender,hasAndroidApp,hasIosApp,daysSinceLastLogin,productsListed,productsSold,productsPassRate,productsWished,productsBought,socialProductsLiked,socialNbFollowers,socialNbFollows,OS,ratiosellbuy,ratiobuysell
2021,Etats-Unis,en,F,False,True,21,9,11,90.0,3,5,31,42,16,iOS,2.200000,0.454545
2484,France,fr,F,False,False,11,4,7,100.0,1,2,48,11,8,Browser,3.500000,0.285714
3215,France,fr,F,False,True,11,33,163,94.0,10,3,60,137,13,iOS,54.333333,0.018405
3498,France,it,M,False,True,14,2,12,83.0,0,1,1,7,8,iOS,12.000000,0.083333
3546,Pays-Bas,en,F,True,False,14,0,18,66.0,14,11,22,71,9,Android,1.636364,0.611111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97171,France,fr,F,False,True,46,0,7,85.0,1,4,52,13,9,iOS,1.750000,0.571429
97480,Etats-Unis,en,F,False,True,24,9,8,100.0,0,1,0,6,8,iOS,8.000000,0.125000
98114,France,fr,F,False,True,15,8,9,100.0,10,1,62,9,10,iOS,9.000000,0.111111
98191,Autriche,de,F,False,True,11,20,20,100.0,80,9,81,17,10,iOS,2.222222,0.450000


In [23]:
df_active.loc[(df_active["ratiobuysell"]>1)&(df_active["ratiobuysell"]!=np.inf)]

,country,language,gender,hasAndroidApp,hasIosApp,daysSinceLastLogin,productsListed,productsSold,productsPassRate,productsWished,productsBought,socialProductsLiked,socialNbFollowers,socialNbFollows,OS,ratiosellbuy,ratiobuysell
588,Royaume-Uni,en,M,True,False,20,0,1,0.0,303,32,6,16,0,Android,0.031250,32.0
960,Hong Kong,en,M,False,True,16,0,1,0.0,223,3,6,5,8,iOS,0.333333,3.0
1054,Pays-Bas,en,F,False,True,377,2,2,100.0,17,8,33,29,8,iOS,0.250000,4.0
1135,Royaume-Uni,en,F,False,True,362,0,5,0.0,47,7,373,16,9,iOS,0.714286,1.4
2453,Italie,it,M,False,True,11,0,1,0.0,3,57,12,28,11,iOS,0.017544,57.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97010,Suède,en,F,True,False,28,2,2,100.0,6,3,27,26,9,Android,0.666667,1.5
97041,Ukraine,fr,F,False,True,11,2,3,0.0,707,18,792,65,157,iOS,0.166667,6.0
98184,France,fr,F,False,False,43,0,1,0.0,24,5,13,5,9,Browser,0.200000,5.0
98368,Portugal,en,F,False,False,11,3,2,0.0,159,17,521,31,10,Browser,0.117647,8.5


In [24]:
df_active.loc[((df_active["ratiosellbuy"]<3)&(df_active["ratiosellbuy"]>0))|((df_active["ratiosellbuy"]<3)&(df_active["ratiobuysell"]>0)),"productsListed"].describe()

count    5313.000000
mean        0.305101
std         3.505064
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max       202.000000
Name: productsListed, dtype: float64

In [25]:
df_active.loc[(df_active["productsListed"]>0)|(df_active["productsSold"]>0)]

,country,language,gender,hasAndroidApp,hasIosApp,daysSinceLastLogin,productsListed,productsSold,productsPassRate,productsWished,productsBought,socialProductsLiked,socialNbFollowers,socialNbFollows,OS,ratiosellbuy,ratiobuysell
46,France,fr,F,False,True,127,1,1,0.0,0,1,0,31,8,iOS,1.0,1.0
133,France,fr,F,False,True,485,1,1,0.0,0,0,6,7,8,iOS,inf,0.0
164,Royaume-Uni,en,F,False,True,665,0,3,66.0,0,0,0,10,8,iOS,inf,0.0
196,Etats-Unis,en,F,False,False,60,2,0,0.0,0,0,7,4,8,Browser,NaN,NaN
242,France,fr,M,False,False,405,0,1,0.0,0,0,0,3,8,Browser,inf,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98623,France,fr,F,False,True,17,1,0,0.0,0,0,0,5,8,iOS,NaN,NaN
98664,France,fr,M,False,True,191,0,1,0.0,8,2,3,3,8,iOS,0.5,2.0
98690,Italie,it,F,True,False,684,0,1,0.0,0,0,1,11,9,Android,inf,0.0
98693,Autriche,de,M,False,True,708,1,0,0.0,0,0,0,5,8,iOS,NaN,NaN


In [26]:
df_active.loc[((df_active["ratiosellbuy"]<1.5)&(df_active["ratiobuysell"]<12)),"productsListed"].describe()

count    357.000000
mean       2.240896
std        6.429672
min        0.000000
25%        0.000000
50%        0.000000
75%        2.000000
max       62.000000
Name: productsListed, dtype: float64

In [33]:
df_active.loc[((df_active["productsSold"]>=1)&(df_active["productsBought"]>=1))]

,country,language,gender,hasAndroidApp,hasIosApp,daysSinceLastLogin,productsListed,productsSold,productsPassRate,productsWished,productsBought,socialProductsLiked,socialNbFollowers,socialNbFollows,OS,ratiosellbuy,ratiobuysell
46,France,fr,F,False,True,127,1,1,0.0,0,1,0,31,8,iOS,1.000000,1.00
588,Royaume-Uni,en,M,True,False,20,0,1,0.0,303,32,6,16,0,Android,0.031250,32.00
960,Hong Kong,en,M,False,True,16,0,1,0.0,223,3,6,5,8,iOS,0.333333,3.00
1054,Pays-Bas,en,F,False,True,377,2,2,100.0,17,8,33,29,8,iOS,0.250000,4.00
1135,Royaume-Uni,en,F,False,True,362,0,5,0.0,47,7,373,16,9,iOS,0.714286,1.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98260,Pays-Bas,en,F,False,False,14,25,20,92.8,0,1,29,8,9,Browser,20.000000,0.05
98352,Espagne,en,F,False,False,26,0,2,0.0,6,1,9,11,8,Browser,2.000000,0.50
98368,Portugal,en,F,False,False,11,3,2,0.0,159,17,521,31,10,Browser,0.117647,8.50
98442,Grèce,en,F,False,True,17,3,1,0.0,9,1,14,31,7,iOS,1.000000,1.00


In [35]:
df_active.loc[((df_active["ratiobuysell"]>12))]

,country,language,gender,hasAndroidApp,hasIosApp,daysSinceLastLogin,productsListed,productsSold,productsPassRate,productsWished,productsBought,socialProductsLiked,socialNbFollowers,socialNbFollows,OS,ratiosellbuy,ratiobuysell
51,Etats-Unis,en,M,False,False,26,0,0,0.0,14,1,1,3,9,Browser,0.0,inf
85,Suède,en,M,False,False,543,0,0,0.0,107,1,0,5,8,Browser,0.0,inf
95,Canada,en,F,True,False,208,0,0,0.0,7,1,1,4,10,Android,0.0,inf
102,Royaume-Uni,en,F,False,False,27,0,0,0.0,0,2,0,3,8,Browser,0.0,inf
114,Etats-Unis,en,M,False,True,15,0,0,0.0,4,6,0,28,29,iOS,0.0,inf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98853,Italie,it,M,False,False,148,0,0,0.0,0,5,7,4,10,Browser,0.0,inf
98865,Allemagne,de,M,False,False,247,0,0,0.0,3,11,6,4,8,Browser,0.0,inf
98879,Danemark,en,F,False,False,204,0,0,0.0,3,1,1,5,8,Browser,0.0,inf
98886,Italie,it,F,False,True,24,0,0,0.0,153,22,13,13,12,iOS,0.0,inf
